In [1]:
import pandas as pd
import numpy as np
import pyreadr
import math
import random
import os
import csv
import matplotlib.pyplot as plt
import warnings
warnings.filterwarnings("ignore")
import time

## Parte I

### Pregunta 1

| Marca – modelo | Precio ($) | Bus Memoria | Memoria | Velocidad de Boost (memoria) | Velocidad Game | Velocidad Bandwidth |
| --- | --- | --- | --- | --- | --- | --- |
| MSI Radeon  | 515 | 128 bits | 8GB GDDR6 | 2602 MHz | 2413 MHz |  | 
| RX 6600 XT |       |         |           |         |          |           |
| BIOSTAR RADEON RX560  | 515 | 128 bits | 4GB DDR5 | 1175 MHz | 1275MHz |  |
| PNY GEFORCE RTX 3060 | 550 | 192 bits | 12GB GDDR6 | 1777 MHz |  | 360 GB | 

Fuentes:
- https://cyccomputer.pe/amd-radeon/21256-msi-radeon-rx-6600-xt-mech-2x-8gb-gddr6-128bits-oc-pn912-v502-015.html
- https://cyccomputer.pe/amd-radeon/20821-biostar-radeon-rx560-4gb-ddr5-128bits-gaming-pnva5615rf41-tbhra-bs2.html
- https://cyccomputer.pe/nvidia-geforce-rtx/21359-pny-geforce-rtx-3060-12gb-gddr6-192bits-xlr8-gaming-oem-pngmr3061n4jcet1bktp.html

Para realizar computación en paralelo, la mejor opción a tomar en cuenta es la MSI Radeon; ya que, cuesta un poco menos que la PNY y tiene varias ventajas, entre estas se tiene que la MSI posee un mayor rendimiento en velocidad de reloj del procesador y de memoria GPU, a pesar de que se está sacrificando un poco de potencia en el procesamiento de la GPU, lo que permite un rendimiento superior para la ejecución de computación en paralelo. 

Por otra parte, la MSI supera en velocidad de la memoria a los otros modelos, con lo cual se incrementa el rendimiento en aplicaciones ya que tienen mayores registros de velocidad a diferencia de la PNY. También, se puede atribuir a ambas un similar funcionamiento de las tarjetas gráficas, ya que ambas poseen una misma anchura del bus de memoria, con lo cual tienen similar capacidad de soportar más datos por ciclo. Finalmente, se puede concluir que la MSI debería ser la tarjeta gráfica por elegir, ya que tiene un precio no muy distante de la PNY y tiene varias características a favor.


### Pregunta 2

Good Data no será reemplazado por Big Data o viceversa, ya que la estructura para obtener la información es diferente. Incluso en Couper (2013) se menciona que las nuevas formas de obtener data son interesantes, pero no se debe descartar los métodos tradicionales como las encuestas. Sin embargo, si se pueden complementar porque ambas forman parte de las herramientas que usa la ciencia de datos para inferir sobre la realidad. O como señala Ng (2022) se debería buscar tener datos de mejor calidad que partan por una selección basada en big data y las técnicas de IA correspondiente que permita mejorar la selección de pequeños datos.

Como ejemplo, se puede entender la información que proviene de las encuestas, ya que son una herramienta potente para levantar información y entender el mundo real, aunque no es la única herramienta. En Couper (2013) se llama una trampa a la idea de pensar que solo las encuestas nos ofrecen la mejor manera de recopilar información. De ahí resulta el Big Data y su facilidad para obtener los datos, los mismos que deben ser parte de un aprendizaje previo y ese redondeo al promedio conocido. Aplicado a la realidad que acabamos de pasar, es el uso de big data por el COVID-19, que finalmente dio lugar a Good data, pues permitió crear modelos de aprendizaje automático referente a trazabilidad del virus u otras aplicaciones de mejora de modelos epidemiológicos.

Otro ejemplo, las empresas en todo el mundo han desbordado sus bancos de datos y se han convertido en big data. Desde las cuentas de redes sociales hasta la cadena de suministro y los datos de gastos, las empresas tienen un acceso ilimitado. Pero con más información no necesariamente mejora la calidad del análisis, ya que la recopilación de datos es solo el primer paso. La gestión de datos, junto con el análisis de big data, lo ayudará a extraer los datos útiles y relevantes de las grandes cantidades de información disponible, y ponerlos en uso para generar valor y productividad para su negocio. Esto último es un claro ejemplo de good data.

### Pregunta 3

Al momento de importar nuestra base de datos, nuestras variables fueron importadas como de tipo 'O'. Inicialmente, tratamos de convertirlas en numéricas para poder realizar cálculos. Sin embargo, tuvimos dificultades realizando ello. Por ende, publicamos nuestro problema en Stackoverflow.

https://stackoverflow.com/questions/72828144/how-can-i-convert-an-object-variable-with-nan-into-an-int-variable-in-python

## Parte II

### Pregunta 1

### Pregunta 2

### Pregunta 3

### Pregunta 4

### Parte a

Primero, importamos los archivos archivos CSV que tenemos en la carpeta data

In [2]:
%%time 

# En folderpath, se debe cambia el path en donde se encuentren los archivos CSV
folderpath =  r"data"
print("Files and directories in a specified path:")
files = []
for filename in os.listdir(folderpath):
    f = os.path.join(folderpath, filename)
    files.append(f)
    if os.path.isfile(f):
        print(f)

Files and directories in a specified path:
data\endes_2021_programas_sociales_hogar.csv
data\endes_2021_RE223132.csv
data\endes_2021_RE516171.csv
data\endes_2021_RE758081.csv
data\endes_2021_REC84DV.csv
data\endes_2021_REC91.csv
data\endes_2021_REC93DVdisciplina.csv
data\endes_2021_REC95.csv
data\endes_2021_RECH23.csv
Wall time: 0 ns


#### Import csv using dask


In [3]:
from dask.distributed import Client, LocalCluster
cluster = LocalCluster()  # Launches a scheduler and workers locally
client = Client(cluster)  # Connect to distributed cluster and override default
client

Client Scheduler: tcp://127.0.0.1:64155 Dashboard: http://127.0.0.1:8787/status,Cluster Workers: 4 Cores: 8 Memory: 12.68 GB


In [4]:
import dask.dataframe as dd

In [5]:
#guardamos los dask dataframes en una lista
dfs = []

for file in files:
    
    df = dd.read_csv(file, dtype=str)
    dfs.append(df)

In [6]:
#asignamos nombres a los dataframes
programas_sociales = dfs[0]
nacimiento = dfs[1]
nupcialidad_fecundidad = dfs[2]
sida_condon = dfs[3]
mortalidad_violencia = dfs[4]
MEF = dfs[5] 
disciplina_infantil = dfs[6]
inmunizacion_salud = dfs[7]
vivienda = dfs[8]

In [7]:
# identificar bases que tengan CASEID
df_list = [programas_sociales, nacimiento, nupcialidad_fecundidad, sida_condon, mortalidad_violencia, MEF, disciplina_infantil,
          inmunizacion_salud,vivienda]

df_list_names = ["programas_sociales", "nacimiento", "nupcialidad_fecundidad", "sida_condon", "mortalidad_violencia", "MEF", "disciplina_infantil",
          "inmunizacion_salud","vivienda"]

df_CASEID = []

for df, df_name in zip(df_list,df_list_names):
       
    if 'CASEID' in df.columns:
        print(df_name)
        
        df_CASEID.append(df)

nacimiento
nupcialidad_fecundidad
sida_condon
mortalidad_violencia
MEF
disciplina_infantil
inmunizacion_salud


#### Merging dfs

In [8]:
base_total = nacimiento.merge(nupcialidad_fecundidad, on = ['CASEID', 'ID1'], how ='left').merge(sida_condon, on = ['CASEID', 'ID1'], how ='left').merge(mortalidad_violencia, on = ['CASEID', 'ID1'], how ='left').merge(MEF, on = ['CASEID', 'ID1'], how ='left').merge(disciplina_infantil, on = ['CASEID', 'ID1'], how ='left').merge(inmunizacion_salud, on = ['CASEID', 'ID1'], how ='left')

In [9]:
df_final = base_total.compute()

In [10]:
df_final.dtypes

ID1         object
CASEID      object
V201        object
V202        object
V203        object
             ...  
S466DC12    object
S466MC12    object
S466YC12    object
S466WC12    object
S466HC12    object
Length: 1272, dtype: object

In [11]:
%%time 

# Seleccionando las variables de interés 
variables = ['ID1', 'SREGION', 'SPROVIN', 'SDISTRI', 'CASEID',
             'S108N', 'S119', 'S119D', 'V218', 'V213', 'V225',    # carac. personales
             'S45D4', 'S45D5', 'S45P4', 'S45P5', 'S45B0',    # vacunas
             'S45B1', 'S45B2', 'S45B3', 'S45F1', 'S45F2',    # vacunas
             'V501', 'V714', 'D104', 'D106', 'D107', 'D108']

data_final = df_final[variables]
data_final.shape

Wall time: 1.12 s


(43137, 27)

In [12]:
for i in variables:
        if data_final[i].dtype == 'O':
            data_final[i] = data_final[i].str.replace(" ", "")
            # Reemplazando los missing values por NaN
            data_final[i] = data_final[i].replace(' ', np.nan)

            # Convirtiendo los datos a valores numéricos
            data_final[i] =  np.floor(pd.to_numeric(data_final[i], errors='coerce').astype('Int64'))

In [13]:
data_final.dtypes

ID1        Float64
SREGION    Float64
SPROVIN    Float64
SDISTRI    Float64
CASEID     Float64
S108N      Float64
S119       Float64
S119D      Float64
V218       Float64
V213       Float64
V225       Float64
S45D4      Float64
S45D5      Float64
S45P4      Float64
S45P5      Float64
S45B0      Float64
S45B1      Float64
S45B2      Float64
S45B3      Float64
S45F1      Float64
S45F2      Float64
V501       Float64
V714       Float64
D104       Float64
D106       Float64
D107       Float64
D108       Float64
dtype: object

#### Creación de por lo menos dos variables

*Las variables de interés son las siguientes:*

D104: Hubo violencia emocional | 0=No, 1=Sí

D106: Experimentó cualquier violencia física menos severa | 0=No, 1=Sí

D107: Experimentó cualquier violencia física severa | 0=No, 1=Sí

D108: Experimentó cualquier violencia sexual | 0=No, 1=Sí



1) Creamos una variable que indique la cantidad de tipos de violencia (emocional, física menos severa, física severa, sexual) que la mujer ha sido víctima por parte de su esposo/compañero. Esta variable está en el rango de 0 a 4.

In [14]:
# Agregando los datos para evaluar cuántas mujeres han sido víctimas de violencia en al menos una de sus modalidades
data_final['violencia'] = data_final['D104'] + data_final['D106'] + data_final['D107'] + data_final['D108']

2) Creamos una variable dicotómica que indique si es que la mujer ha sido víctima de, al menos, un tipo de violencia por parte de su esposo/compañero

In [15]:
# Creando variable dummy 
data_final['y'] = data_final['violencia']
data_final.loc[(data_final['y'] > 0), 'y'] = 1

#### Dos estadísticos descriptivos 

In [16]:
tab1 = data_final['violencia'].value_counts().to_frame().reset_index()
tab1.rename(columns = {'index':'n_violencia', 'violencia':'n_casos'}, inplace = True)
tab1['n_violencia'] = tab1['n_violencia'].astype(int)

tab1['%'] = round( (tab1['n_casos']/sum(tab1.n_casos))*100 , 2)
tab1

,n_violencia,n_casos,%
0,0,18395,66.16
1,1,4372,15.72
2,2,2621,9.43
3,3,1598,5.75
4,4,817,2.94


Se observa que el 66.16% de mujeres no sufren ningún tipo de violencia por parte de su esposo/compañero. Sin embargo, hay un 15.72% de mujeres que sufren 1 tipo de violencia. Se encuentra que solo el 2.94% de mujeres sufren los 4 tipos de violencia.

In [17]:
tab2 = data_final['y'].value_counts().to_frame().reset_index()
tab2.rename(columns = {'index':'d_violencia', 'y':'n_casos'}, inplace = True)
tab2['d_violencia'] = tab2['d_violencia'].astype(int)

tab2['%'] = round( (tab2['n_casos']/sum(tab2.n_casos))*100 , 2)
tab2

,d_violencia,n_casos,%
0,0,18395,66.16
1,1,9408,33.84


Se encuentra que un 33.84% de las mujeres de la muestra sufren de, al menos, algún tipo de violencia por parte de su esposo/compañero.

#### Gráficos

In [ ]:
tab_plot=tab2.rename(columns = {'d_violencia':'Victima_control_pareja', 'n_casos':'Total'})
tab_plot['Victima_control_pareja'] = tab_plot.Victima_control_pareja.astype(str)


fig = plt.figure(figsize = (10, 5))
#  Bar plot
plt.bar(tab_plot['Victima_control_pareja'], tab_plot['Total'], color ='paleturquoise',
        width = 0.5)
plt.xlabel("Víctima de control por parte de su pareja")
plt.ylabel("Encuestadas")
plt.title("Número de encuestadas víctimas de control por parte de su pareja")
plt.show()

In [ ]:
tot_D106 = data_final.groupby('D106')['D106'].count()
tot_D106=tot_D106.rename(None).to_frame().reset_index()
tot_D106.columns=['Victima','Violencia_menos_severa']

tot_D107 = data_final.groupby('D107')['D107'].count()
tot_D107=tot_D107.rename(None).to_frame().reset_index()
tot_D107.columns=['Victima','Violencia_severa']

tot_D108 = data_final.groupby('D108')['D108'].count()
tot_D108=tot_D108.rename(None).to_frame().reset_index()
tot_D108.columns=['Victima','Violencia_sexual']

tab_plot = pd.merge(tot_D106, tot_D107, on='Victima', how='left')
tab_plot = pd.merge(tab_plot, tot_D108, on='Victima', how='left')
tab_plot['Victima'] = tab_plot.Victima.astype(str)

tab_plot = tab_plot.set_index('Victima')

In [ ]:
tab_plot

In [ ]:
tab_plot.plot(kind="bar",figsize=(15, 8), color = ['lightcoral','indianred','firebrick'])

plt.title("Número de mujeres víctima de violencia, según el tipo de violencia")

plt.xlabel("Total")

plt.ylabel("Experimentó violencia")

### Parte b

In [ ]:
data_final.columns

#### Definición de predictores (X) y vector de target (y)

Nuestro variable a predecir (y) será una una dummy que toma el valor de 1 si es que la mujer ha sufrido algún tipo de violencia por parte de su compañero/esposo. Esta variable toma el valor de 0 si es que no ha sufrido violencia.

En cuanto a las variables que se usarán como predictores (X), serán las siguientes:

* SREGION: Región Natural (Categórica)
* S108N: Nivel educativo (Categórica)
* S119: Idioma o lengua materna (Categórica)
* S119D: Etnicidad (Categórica)
* V213: Está embarazada (Dummy)
* V501: Estado conyugal actual (Categórica)
* V714: Entrevistada actualmente trabaja  (Dummy)
* V218: Número de niños con vida (Continua)

Dado que no es intención del ejercicio complejizar el modelo, solo dichas variables fueron consideradas para el modelo de ML que será utilizado.

#### Limpieza de predictores a ser utilizados en el modelo de ML

In [ ]:
categorical_vars = ['SREGION', 'S108N', 'S119', 'S119D', 'V213', 'V501', 'V714'] 

integer_vars = ['V218']

final_vars = ['SREGION', 'S108N', 'S119', 'S119D', 'V213', 'V501', 'V714', 'V218', 'y']

In [ ]:
#limitamos nuestro dataframe a solo las variables que serán utilizadas en el modelo de ML
data_ml = df_final[final_vars]

In [ ]:
#convertimos las variables que deberian ser categóricas en "category"
for col in categorical_vars:
    data_ml[col] = data_ml[col].astype('category',copy=False)

In [ ]:
#colocandole labels a las variables categóricas para mayor entendimiento
#Tambien reducimos el número de categorías para determinadas variables

#región natural:
data_ml.SREGION.replace({'1':'lima', '2': 'costa', '3':'sierra','4':'selva'}, inplace = True)

#nivel educativo:
data_ml.S108N.replace({'0':'ninguno', '1': 'primaria', '2':'secundaria','3':'superior','4':'superior','5':'superior'}, inplace = True)

#lengua materna:
data_ml.S119.replace({'1':'nativo', '2': 'nativo', '3':'nativo','4':'nativo','5':'nativo','6':'nativo', '7':'nativo', '8':'nativo', '9':'nativo', '10':'castellano', '11':'extranjero', '12':'extranjero'}, inplace = True)

#etnicidad:
data_ml.S119D.replace({'1':'nativo', '2': 'nativo', '3':'nativo','4':'nativo','5':'negro','6':'blanco', '7':'mestizo', '8':'otro', '98':'no_sabe'}, inplace = True)

#está embaraza:
data_ml.V213.replace({'1':'si', '0': 'no'}, inplace = True)

#estado conyugal
data_ml.V501.replace({'0':'no_casada', '1': 'casada', '2':'conviviente', '3':'viuda', '4':'divorciada','5':'no_viven_juntos'}, inplace = True)

#actualmente trabaja
data_ml.V714.replace({'1':'si', '0': 'no'}, inplace = True)

In [ ]:
#verificamos las categorías
for col in categorical_vars:
    print(data_ml[col].value_counts())

#### Aplicando encoding en las variables categóricas 

In [ ]:
import sklearn
#!pip install category_encoders
import category_encoders as ce

In [ ]:
ce_OHE = ce.OneHotEncoder(cols=categorical_vars)

data_final = ce_OHE.fit_transform(data_ml)
data_final.head()

In [ ]:
data_final.columns

#### Aplicando modelo de ML

In [ ]:
from sklearn.linear_model import *
from sklearn.pipeline import *
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.preprocessing import LabelEncoder
from sklearn.pipeline import *

In [ ]:
data_final = data_final[data_final['y'].notna()]
